In [36]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
#try:
#    os.chdir("L:\laupodteam\AIOS\Huibert-Jan\Celldynclustering\celldyn_embedder")
#except FileNotFoundError:
#    os.chdir('C:/Users/Huibert-Jan/Documents/Werk/UMCU/celldyn_embedder')    


## overhead
import sys
import os
import gc
from tqdm import tqdm

## data structures
import pandas as pd
import numpy as np

## plotting
import matplotlib.pyplot as plt
import seaborn as sns

## embedding
import umap

## model selection and tuning
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedGroupKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

## models
from xgboost import XGBRegressor, XGBRFClassifier
from sklearn.linear_model import Lasso
from statsmodels.regression.linear_model import OLS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### load imputed data

In [4]:
# "/media/UMCU/data/CellDyn/celldyn.feather"
current_directory = os.path.dirname(os.path.realpath("."))
os.chdir("T:\\laupodteam\\AIOS\\Bram\\data\\CellDyn")

In [15]:
celldyn = pd.read_feather("celldyn_FULL_transformed_df.feather")
celldyn.columns = celldyn.columns.str.lower()
gc.collect()

323

### select independent variables and age

In [17]:
meas_cols = [c for c in celldyn if ('combo' in c) or ('c_b_' in c)]

In [38]:
X = celldyn[meas_cols].to_numpy()
y = celldyn.age.to_numpy()

In [31]:
from sklearn.base import BaseEstimator
class RegrSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = Lasso(),
    ):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    # def predict_proba(self, X):
    #     return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [42]:
parameters = [
    {
        'regr__estimator': [Lasso()],
        
    },
    {
        'regr__estimator': [XGBRegressor()],
    },
]


In [40]:
pipe = Pipeline([
    ("scaler",StandardScaler()),
    ("regr",RegrSwitcher())
])

X_train,X_test, y_train,y_test = train_test_split(X,y)

In [43]:
gscv = GridSearchCV(pipe, parameters, cv=10, n_jobs=-1, return_train_score=False, verbose=3)
gscv.fit(X_train, y_train)

Fitting 10 folds for each of 2 candidates, totalling 20 fits
